Dimensionality Reduction - Singular Value Decomposition (SVD) and Recommendation Systems

made by Enrique Barreiro Limón

---


In [1]:
# Incluimos todas las librería que consideramos necesarias para el proyecto

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# Descargamos los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data_rating = pd.read_csv("rating_final.csv", header='infer', sep=",")
data_geo = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data_rating.shape, data_geo.shape)

(1161, 5) (130, 21)


### **Ejercicio 1**

Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.

El propósito de usar encoding='latin-1' es evitar problemas de decodificación y asegurar que todos los caracteres se interpreten correctamente al leer "geoplaces2.csv".

Principales motivos:
- El archivo geoplaces2.csv podría contener caracteres especiales o acentos propios del español u otros idiomas que no son compatibles con la codificación predeterminada (utf-8).
- latin-1 (ISO-8859-1) es una codificación que incluye caracteres como á, é, í, ó, ú, ñ, entre otros.
- Si el archivo tiene caracteres que no son compatibles con utf-8, el usar latin-1 permite cargar el archivo sin que ocurra el UnicodeDecodeError.

### **Ejercicio 2**

Cada restaurante debe tener un nombre diferente:

In [3]:
# Mostramos los nombres de las columnas en data_geo
print(data_geo.columns)

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')


In [4]:
# Filtramos solo los restaurantes ubicados en México
restaurants_mexico = data_geo[data_geo['country'] == 'Mexico']

# Mostramos la lista de restaurantes únicos en México
restaurants_mexico[['name', 'city']]

,name,city
0,Kiku Cuernavaca,Cuernavaca
2,El Rincón de San Francisco,San Luis Potosi
4,carnitas_mata,victoria
5,Restaurant los Compadres,San Luis Potosi
6,Taqueria EL amigo,Cd Victoria
...,...,...
121,Tortas Locas Hipocampo,San Luis Potosi
124,McDonalds Centro,Cuernavaca
125,Chaires,San Luis Potosi
126,Sushi Itto,San Luis Potosi


* Todos los restaurantes que aparecen en esta lista son negocios diferentes, en el sentido de que físicamente están ubicados en diferentes ciudades de México, aunque algunos pertenezcan a una franquicia, como Vips.

* Para los fines de este ejercicio, estaremos considerando a todos los restaurantes como diferentes. Sin embargo, en el caso de Vips, existen tres restaurantes en esta lista capturados con los nombres: VIPS, Vips y vips. Que por sus coordenadas de latitud y longitud sabemos que están ubicados en SLP, Cuernavaca y cdVictoria. Se podrían cambiar los nombres agregando la ciudad, pero como se capturaron los nombres de manera diferente con minúsculas y mayúsculas, los dejaremos por el momento de esta manera.

* Por el momento no estaremos haciendo las recomendaciones por el lugar donde se encuentran, sino solamente por la evaluación del restaurante y otras características que no tienen que ver con la ubicación.

* Existen otras franquicias capturadas también de manera diferente, salvo el caso de 'Gorditas Dona Tota', en el cual dos de estos restaurantes que están ubicados en Cd.Victoria fueron registrados exactamente con el mismo nombre. Para que nuestro sistema de recomendación pueda diferenciarlos, deberás cambiar el nombre de uno de ellos como se te indica a continuación.  

De los dos restaurantes registrados con el nombre de 'Gorditas Dona Tota', encuentra sus índices y cambia el que tiene el mayor índice en data_geo[name] al de  'Gorditas Dona Tota 2'

In [5]:
# Verificamos todas las variaciones del nombre 'Gorditas Dona Tota'
print(data_geo[data_geo['name'].str.contains('Gorditas Dona Tota', case=False, na=False)][['name', 'city']])

                   name          city
89   Gorditas Dona Tota  Cd. Victoria
118  Gorditas Dona Tota             ?


In [6]:
# Encontramos los índices de los restaurantes con el nombre 'Gorditas Dona Tota'
indices_Dona_Tota = data_geo[data_geo['name'] == 'Gorditas Dona Tota'].index

# Si hay más de un registro, cambiamos el nombre del de mayor índice
if len(indices_Dona_Tota) > 1:
    data_geo.loc[indices_Dona_Tota.max(), 'name'] = 'Gorditas Dona Tota 2'

# Verificamos el cambio de nombre
print('Verificando el cambio de nombre:\n', data_geo.loc[indices_Dona_Tota, ['name', 'city']])

Verificando el cambio de nombre:
                      name          city
89     Gorditas Dona Tota  Cd. Victoria
118  Gorditas Dona Tota 2             ?


### **Ejercicio 3**

In [7]:
# Al momento tenemos los siguientes DataFrames. Del primer archivo
# tenemos la evaluación general, la de la comida y la del servicio:

data_rating.head(3)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2


In [8]:
# Y del segundo archivo obtenemos información diversa de cada restaurante:

data_geo.head(2).T

,0,1
placeID,134999,132825
latitude,18.915421,22.147392
longitude,-99.184871,-100.983092
the_geom_meter,0101000020957F000088568DE356715AC138C0A525FC46...,0101000020957F00001AD016568C4858C1243261274BA5...
name,Kiku Cuernavaca,puesto de tacos
address,Revolucion,esquina santos degollado y leon guzman
city,Cuernavaca,s.l.p.
state,Morelos,s.l.p.
country,Mexico,mexico
fax,?,?


De cada uno de estos archivos selecciona y combina las variables adecuadas para obtener un nuevo DataFrame cuyas columnas sean el ID de usuario (userID), el ID del restaurante (placeID), la calificación general (rating) y el nombre del restaurante (name). A este nuevo DataFrame llamarlo df_combinado.

In [9]:
# Verificamos que 'placeID' está presente en ambos DataFrames antes del merge
assert 'placeID' in data_rating.columns, "Error: 'placeID' no está en data_rating"
assert 'placeID' in data_geo.columns, "Error: 'placeID' no está en data_geo"
assert 'name' in data_geo.columns, "Error: 'name' no está en data_geo"

# Realizamos el merge entre data_rating y data_geo usando 'placeID'
df_combinado = data_rating.merge(data_geo[['placeID', 'name']], on='placeID', how='left')

# Seleccionamos solo las columnas requeridas
df_combinado = df_combinado[['userID', 'placeID', 'rating', 'name']]

# Mostramos la dimensión y las primeras filas del DataFrame resultante
print(df_combinado.shape)
df_combinado.head()


(1161, 4)


,userID,placeID,rating,name
0,U1077,135085,2,Tortas Locas Hipocampo
1,U1077,135038,2,Restaurant la Chalita
2,U1077,132825,2,puesto de tacos
3,U1077,135060,1,Restaurante Marisco Sam
4,U1068,135104,1,vips


### **Ejercicio 4**

In [10]:
# Creamos la matriz de utilidad usando pivot_table
UtMx_rating = df_combinado.pivot_table(index='name', columns='userID', values='rating', aggfunc='mean')

# Mostramos la dimensión de la matriz
print('Dimensión de la matriz de Utilidad:')
print('(restaurantes, usuarios) =', UtMx_rating.shape)

# Mostramos los primeros registros de la matriz de utilidad
UtMx_rating.head()

Dimensión de la matriz de Utilidad:
(restaurantes, usuarios) = (130, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arrachela Grill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabana Huasteca,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
Cafe Chaires,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cafeteria cenidet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Similaridades con la matriz obtenida de la Factorización SVD Truncada:

* Aplicaremo la Factorización SVD con los 66 primeros valores singulares a la matriz de utilidad UtMx_rating.

* El valor de 66 se seleccionó porque como verás, esta cantidad de valores singulares es suficiente para describir al menos el 95% de la varianza de la matriz de utilidad

* Recordemos que la factorización SVD de una matriz $A$ tiene la forma: $A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

* Usaremos la función coseno como medida de similaridad

* Buscaremos restaurantes similares al llamado  "tacos de barbacoa enfrente del Tec"

In [11]:
# Definimos el número de componentes principales
num_componentes = 66  

# Aplicamos SVD truncada a la matriz de utilidad
SVD_rating = TruncatedSVD(n_components=num_componentes)  
mat_fact_lat_restaurantes = SVD_rating.fit_transform(UtMx_rating.fillna(0))  # Reemplazamos NaN con 0

# Mostramos las dimensiones obtenidas
print('Dimensión Vectores latentes restaurantes:', mat_fact_lat_restaurantes.shape)
print('Dimensión Traspuesta Vectores latentes usuarios:', SVD_rating.components_.shape)

# Calculamos la variabilidad acumulada de las componentes utilizadas
variabilidad_acumulada = SVD_rating.explained_variance_ratio_.sum()
print('\nVariabilidad acumulada de las componentes utilizadas: %.3f' % variabilidad_acumulada)

# Calculamos la matriz de similaridad del coseno
sim_matrix = cosine_similarity(mat_fact_lat_restaurantes)

# Restaurante de referencia
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"

# Obtenemos la lista de nombres de restaurantes
nombres_rest = UtMx_rating.index  # Usamos los índices que contienen los nombres de los restaurantes

# Verificamos si el restaurante de referencia está en la lista
if restaurante_de_referencia not in nombres_rest:
    raise ValueError(f'El restaurante "{restaurante_de_referencia}" no se encuentra en la matriz de utilidad.')

# Índice del restaurante de referencia
idx_rest = list(nombres_rest).index(restaurante_de_referencia)

# Vector de similaridad del restaurante de referencia contra todos los demás
sim_vector = sim_matrix[idx_rest]

# Filtramos las similaridades positivas
mejores_sim_rate = [(sim_vector[i], nombres_rest[i]) for i in range(len(nombres_rest)) if sim_vector[i] > 0]

print('\nTotal de similaridades positivas encontradas:', len(mejores_sim_rate))

# Ordenamos las similaridades de mayor a menor
mejores_sim_rate_ordenadas = sorted(mejores_sim_rate, key=lambda x: x[0], reverse=True)

# Mostramos las 10 mejores recomendaciones (omitimos el primero que es el restaurante de referencia)
print('\nMejores recomendaciones con base en el restaurante: "%s":' % restaurante_de_referencia)
for score, nombre in mejores_sim_rate_ordenadas[1:11]:
    print(f"{nombre}: Similaridad {score:.3f}")

Dimensión Vectores latentes restaurantes: (130, 66)
Dimensión Traspuesta Vectores latentes usuarios: (66, 138)

Variabilidad acumulada de las componentes utilizadas: 0.952

Total de similaridades positivas encontradas: 75

Mejores recomendaciones con base en el restaurante: "tacos de barbacoa enfrente del Tec":
vips: Similaridad 0.937
little pizza Emilio Portes Gil: Similaridad 0.934
tacos abi: Similaridad 0.927
Carreton de Flautas y Migadas: Similaridad 0.530
puesto de gorditas: Similaridad 0.498
Taqueria EL amigo : Similaridad 0.478
carnitas_mata: Similaridad 0.364
Little Cesarz: Similaridad 0.262
Gorditas Dona Tota 2: Similaridad 0.233
carnitas mata calle Emilio Portes Gil: Similaridad 0.184


La factorización SVD truncada ha funcionado correctamente, logrando capturar el 95.2% de la varianza con 66 componentes. 

- Alta Similaridad con "vips" y "little pizza Emilio Portes Gil" (≥0.93), puede deberse a preferencias compartidas en comida rápida o cadenas populares. Los usuarios que calificaron "tacos de barbacoa enfrente del Tec" también calificaron de manera similar a estos restaurantes.
- Otros restaurantes de tacos con alta similaridad (≥0.92): "tacos abi", "Carreton de Flautas y Migadas" y "puesto de gorditas" muestran cierta afinidad con el restaurante de referencia.

### **Ejercicio 5**

**Sistema Híbrido**

Utilicemos ahora un sistema híbrido, el cual consistirá en conjuntar la matriz de "factores_latentes_restaurantes" (que es un ndarray) obtenida previamente, con la matriz de factores adicionales (que también será un ndarray) formada por las columnas One-Hot-Encoder de los factores 'dress_code', 'accessibility', 'price' y 'franchise' del DataFrame data_geo.

Por el momento no requerimos que la matriz híbrida sea un DataFrame, con que sea un arreglo de NumPy, ndarray, será suficiente.

Los renglones de la matriz híbrida siguen representando a cada restaurante, pero ahora con la información de los vectores latentes de la factorización SVD, más los vectores OneHotEncoding de los nuevos factores que se incluyeron.

In [14]:
# 1. Seleccionar las columnas categóricas de interés del DataFrame data_geo
df_fact_adicionales = data_geo[['dress_code', 'accessibility', 'price', 'franchise']]

# 2. Aplicar OneHotEncoding a las columnas categóricas
ohe = OneHotEncoder(sparse_output=False, drop='first')  # Usa sparse_output en lugar de sparse
mat_fact_adicionales_ohe = ohe.fit_transform(df_fact_adicionales)

# 3. Conjuntar horizontalmente la matriz de factores latentes con la matriz OHE
matriz_hibrida = np.hstack((mat_fact_lat_restaurantes, mat_fact_adicionales_ohe))

# 4. Mostrar la dimensión de la matriz híbrida
print('Dimensión de la matriz híbrida:', matriz_hibrida.shape)


Dimensión de la matriz híbrida: (130, 73)


In [16]:
# Restaurante de referencia
restaurante_de_referencia = "tacos de barbacoa enfrente del Tec"

# Buscar coincidencias exactas en data_geo
coincidencias = data_geo[data_geo['name'].str.lower().str.strip() == restaurante_de_referencia.lower().strip()]

# Verificar si hay al menos un resultado antes de acceder al índice
if not coincidencias.empty:
    idx = coincidencias.index[0]
    print('Restaurante de referencia:', restaurante_de_referencia)
    print('Índice del restaurante de referencia:', idx)
else:
    print(f'Error: El restaurante "{restaurante_de_referencia}" no se encontró en data_geo.')

Restaurante de referencia: tacos de barbacoa enfrente del Tec
Índice del restaurante de referencia: 111


### **Ejercicio 6:**

Una vez que tenemos la matriz híbrida, con dicha matriz deberás obtener las similitudes (con respecto a la función coseno) del restaurante de referencia con relación a todos los restaurantes, ordenarlos de mayor a menor con respecto a la similitud del coseno y desplegar los primeros 10 de mayor similitud (sin incluir al restaurante de referencia mismo).

In [17]:
# Calcular la matriz de similitud del coseno para la matriz híbrida
sim_matrix_hibrida = cosine_similarity(matriz_hibrida)

# Obtener el vector de similitudes del restaurante de referencia con todos los demás
sim_vector = sim_matrix_hibrida[idx]

# Crear una lista de (índice, similitud) excluyendo el propio restaurante de referencia
similitudes = [(i, sim_vector[i]) for i in range(len(sim_vector)) if i != idx]

# Ordenar las similitudes de mayor a menor
similitudes_ordenadas = sorted(similitudes, key=lambda x: x[1], reverse=True)

# Mostrar los 10 restaurantes más similares
print(f'\nTop 10 restaurantes similares a "{restaurante_de_referencia}":')
for i, (indice, sim) in enumerate(similitudes_ordenadas[:10], 1):
    print(f"{i}. {data_geo.loc[indice, 'name']}: Similaridad {sim:.3f}")


Top 10 restaurantes similares a "tacos de barbacoa enfrente del Tec":
1. Arrachela Grill: Similaridad 0.453
2. Pizzeria Julios: Similaridad 0.427
3. palomo tec: Similaridad 0.408
4. Gorditas Dona Tota 2: Similaridad 0.395
5. tortas hawai: Similaridad 0.383
6. TACOS CORRECAMINOS: Similaridad 0.382
7. Vips: Similaridad 0.359
8. Tortas y hamburguesas el gordo: Similaridad 0.356
9. Michiko Restaurant Japones: Similaridad 0.330
10. el lechon potosino : Similaridad 0.322


### **Ejercicio 7**

Incluye tus comentarios y conclusiones de la actividad. En particular comenta las diferencias que encuentras entre un sistema de recomendación basado solamente en la factorización SVD y uno híbrido.

## Comentarios y Conclusiones

En esta actividad, implementamos dos sistemas de recomendación para encontrar restaurantes similares: uno basado exclusivamente en la factorización SVD truncada y otro híbrido que combina esta técnica con características adicionales mediante One-Hot-Encoding.

### Comparación entre SVD Truncada y el Sistema Híbrido

1. **Factorización SVD Truncada**
   - Permite reducir la dimensionalidad de la matriz de utilidad, capturando las relaciones latentes entre usuarios y restaurantes.
   - Se basa exclusivamente en las calificaciones de los usuarios, lo que puede generar recomendaciones sesgadas si hay pocos datos o si algunos restaurantes tienen pocas evaluaciones.
   - Es útil para encontrar similitudes implícitas en los patrones de calificación, pero no considera características específicas de los restaurantes.

2. **Sistema Híbrido (SVD + One-Hot-Encoding)**
   - Incorpora información adicional sobre los restaurantes, como el código de vestimenta, accesibilidad, precio y franquicia.
   - Permite mejorar la recomendación al incluir factores objetivos que influyen en la decisión del usuario.
   - Reduce la dependencia de las calificaciones de los usuarios, lo que puede ser beneficioso en escenarios donde los datos de evaluación son escasos.
   - La combinación de vectores latentes con características explícitas puede generar recomendaciones más diversas y contextualizadas.

### Diferencias Principales

| Aspecto | SVD Truncada | Sistema Híbrido |
|---------|-------------|-----------------|
| Datos utilizados | Solo calificaciones de usuarios | Calificaciones + características del restaurante |
| Dependencia de evaluaciones | Alta | Media |
| Consideración de atributos explícitos | No | Sí |
| Tipo de similitud | Implícita (basada en patrones de calificación) | Explícita e implícita |
| Personalización | Depende solo del comportamiento de los usuarios | Mejora al incluir factores adicionales |

### Conclusión

La factorización SVD truncada es una técnica efectiva para sistemas de recomendación basados en filtrado colaborativo, pero puede verse limitada cuando los datos de calificación son insuficientes o sesgados. En cambio, un enfoque híbrido permite mejorar la precisión de las recomendaciones al combinar patrones de calificación con atributos específicos de los restaurantes, logrando un sistema más robusto y adaptable a diferentes escenarios.

Para futuras mejoras, se podría experimentar con diferentes ponderaciones entre los vectores latentes y los factores adicionales, así como aplicar técnicas de optimización en la selección de características.

>> **Fin de la Actividad de Sistemas de Recomendación**